# Make Transaction

Implementazione di funzioni che dal dataframe contenente gli eventi e dopo essere stata applicata la feature redaction crea le **transazioni** per ogni partecipante.

In [ ]:
import nbimporter
from t1_ETLBasics import calories_to_df,sleep_to_df,exercise_to_df
from t3_TSToEvents import calories_to_events, sleep_to_events,exercise_to_events
from t4_FeatureRedaction import calories_kmeans, sleep_kmeans, exercise_kmeans
import dateutil as du
from datetime import datetime
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
from operator import itemgetter
PATH = '../../pmdata/'

## Calories

In [ ]:
cal = calories_kmeans(calories_to_events(calories_to_df(PATH,[1,2])))

In [ ]:
calories = cal.copy()

In [ ]:
calories

In [ ]:
def get_cal_transaction(df):
    """Funzione che crea le transazioni delle calorie per ogni partecipante
    
    Parametri:
    - df: dataframe da cui creare le transazioni
    
    Ritorna:
    - df_trans: lista contenente le transazioni per ogni partecipante"""
    df['label'] = df.apply(lambda x:   f'calories cluster: {x.cluster_index.values[0]}, weekend: {x.weekend.values[0]}', axis = 1)
    df['t_item'] = df.apply(lambda x: [datetime.strptime(x.ts.values[0], '%Y-%m-%d %H'), x.label.values[0]], axis=1)
    df['vt']  = [du.parser.parse(x[1]).strftime('%Y-%m-%d') for x in list(df.index)]
    df_trans =  list(df[['id','vt','t_item','ts']].groupby(['id'])['t_item'].apply(list))
    #return df_trans[0]
    return df_trans

In [ ]:
cal_transaction = get_cal_transaction(calories)

In [ ]:
cal_transaction

## Sleep

In [ ]:
sl = sleep_kmeans(sleep_to_events(sleep_to_df(PATH,[1])))

In [ ]:
sleep = sl.copy()

In [ ]:
sleep

In [ ]:
def get_sleep_transaction(df):
    """Funzione che crea le transazioni per sleep per ogni partecipante
    
    Parametri:
    - df: dataframe da cui creare le transazioni
    
    Ritorna:
    - df_agg: lista contenente le transazioni per ogni partecipante"""
    df['label'] = df.apply(lambda x:   f'Sleep: weekend: {x.weekend}, efficiency: {x.efficiency}, durationSleep: {x.durationH}', axis = 1)
    df['t_item'] = df.apply(lambda x: [datetime.strptime(x.ts, '%Y-%m-%d %H'), x.label], axis=1)
    df['vt']  = [du.parser.parse(x[1]).strftime('%Y-%m-%d') for x in list(df.index)]
    #df_agg= list(df[['pid','vt','t_item','ts']].groupby(['pid','vt'])['t_item'].apply(list))
    df_agg= list(df[['pid','vt','t_item','ts']].groupby(['pid'])['t_item'].apply(list))
    #return [ item for elem in df_agg for item in elem]
    return df_agg
    
    

In [ ]:
sleep_transaction = get_sleep_transaction(sleep)

In [ ]:
sleep_transaction

## Exercise

In [ ]:
ex = exercise_kmeans(exercise_to_events(exercise_to_df(PATH,[1,2])),4)

In [ ]:
exercise = ex.copy()

In [ ]:
exercise

In [ ]:
def get_exercise_transaction(df):
    """Funzione che crea le transazioni degli exercise per ogni partecipante
    
    Parametri:
    - df: dataframe da cui creare le transazioni
    
    Ritorna:
    - df_agg: lista contenente le transazioni per ogni partecipante"""
    df['label'] = df.apply(lambda x:   f'exercise cluster: {x.cluster_index}, part day: {x.part_day}, Activity: {x.activityName}', axis = 1)
    df['t_item'] = df.apply(lambda x: [datetime.strptime(x.ts, '%Y-%m-%d %H'), x.label], axis=1)
    df['vt']  = [x[1].strftime('%Y-%m-%d') for x in list(df.index)]
    #df_agg = list(df[['pid','vt','t_item']].groupby(['pid','vt'])['t_item'].apply(list))
    df_agg = list(df[['pid','vt','t_item']].groupby(['pid'])['t_item'].apply(list))
    return df_agg
    #return [ item for elem in df_agg for item in elem]
    

In [ ]:
exercise_transaction = get_exercise_transaction(exercise)

In [ ]:
exercise_transaction

In [ ]:
def get_master(t1,t2,t3):
    """Funzione che unisce le transazioni dei file.
    
    Parametri:
    - t1: transazioni calories
    - t2: transazione sleep
    - t3: transazioni exercise
    
    Ritorna:
    - una lista contenente le transazioni insieme e ordinate per timestamp"""
    t= t1+t2+t3
    return sorted(t,key=itemgetter(0))
    

In [ ]:
def get_master_transaction(t1,t2,t3):
    """Funzione che genera la lista contenente le transazioni ordinate per ogni partecipante
    
    Parametri:
    - t1: transazioni calories
    - t2: transazione sleep
    - t3: transazioni exercise
    
    Ritorna:
    - t: lista che per ogni elemento ha la lista delle transazioni per il partecipante i-esimo"""
    t = []
    for i in range(len(t1)):
        t.append(get_master(t1[i],t2[i],t3[i]))
    return t

In [ ]:
t = get_master_transaction(cal_transaction,sleep_transaction,exercise_transaction)

In [ ]:
t